In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [23]:
df1 = client.to_pandas("""
SELECT *
FROM oso.int_sbom_from_ossd
WHERE
  package_artifact_name = 'viem'
  AND dependent_artifact_id = 'oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA='
ORDER BY snapshot_at
""")
df1

,dependent_artifact_id,dependent_artifact_source,dependent_artifact_namespace,dependent_artifact_name,package_artifact_id,package_artifact_source,package_artifact_namespace,package_artifact_name,package_artifact_url,package_version,snapshot_at
0,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.21.38,2025-04-11 10:02:57.000
1,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.21.38,2025-02-03 23:58:22.000
2,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.7.13,2025-02-03 23:58:22.000
3,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.21.38,2025-01-25 21:33:29.000
4,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.7.13,2025-08-27 04:47:12.000
...,...,...,...,...,...,...,...,...,...,...,...
189,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.7.13,2025-03-25 17:46:22.000
190,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.21.32,2025-04-01 13:22:05.000
191,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.21.38,2025-01-29 20:13:54.000
192,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,GITHUB,coinbase,paymaster-bundler-examples,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,NPM,,viem,https://www.npmjs.com/package/viem,2.21.38,2025-05-02 07:41:57.000


In [27]:
df2 = client.to_pandas("""

WITH base_raw AS (
  SELECT
    dependent_artifact_id,
    package_artifact_id,
    snapshot_at,
    MAX(package_version) AS package_version -- super basic semver
  FROM oso.int_sbom_from_ossd
  WHERE
    package_artifact_name = 'viem'
    AND dependent_artifact_id = 'oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA='
  GROUP BY 1,2,3
),
-- Keep only the LAST snapshot per month per dependent+package+version
monthly_base AS (
  SELECT * FROM (
    SELECT
      br.*,
      DATE_TRUNC('month', br.snapshot_at) AS month_bucket,
      ROW_NUMBER() OVER(
        PARTITION BY br.dependent_artifact_id,br.package_artifact_id,br.package_version,DATE_TRUNC('month', br.snapshot_at)
        ORDER BY br.snapshot_at DESC
      ) AS rn_month
    FROM base_raw br
  )
  WHERE rn_month=1
),
dependent_snapshots AS (
  SELECT DISTINCT dependent_artifact_id,snapshot_at FROM monthly_base
),
dependent_snapshots_seq AS (
  SELECT
    dependent_artifact_id,
    snapshot_at,
    LEAD(snapshot_at) OVER(PARTITION BY dependent_artifact_id ORDER BY snapshot_at) AS next_snapshot_at
  FROM dependent_snapshots
),
presence AS (
  SELECT DISTINCT
    dependent_artifact_id,
    package_artifact_id,
    package_version,
    snapshot_at
  FROM monthly_base
),
presence_seq AS (
  SELECT
    p.*,
    ROW_NUMBER() OVER(PARTITION BY dependent_artifact_id,package_artifact_id ORDER BY snapshot_at) AS rn,
    LAG(package_version) OVER(PARTITION BY dependent_artifact_id,package_artifact_id ORDER BY snapshot_at) AS prev_version
  FROM presence p
),
adds AS (
  SELECT
    'ADD_DEPENDENCY' AS event_type,
    snapshot_at AS event_time,
    dependent_artifact_id,
    package_artifact_id,
    CAST(NULL AS varchar) AS version_before,
    package_version AS version_after
  FROM presence_seq
  WHERE rn=1
),
upgrades AS (
  SELECT
    'UPGRADE_DEPENDENCY' AS event_type,
    snapshot_at AS event_time,
    dependent_artifact_id,
    package_artifact_id,
    prev_version AS version_before,
    package_version AS version_after
  FROM presence_seq
  WHERE prev_version IS NOT NULL AND package_version <> prev_version
),
last_presence AS (
  SELECT
    dependent_artifact_id,
    package_artifact_id,
    MAX(snapshot_at) AS last_seen_at,
    MAX_BY(package_version,snapshot_at) AS last_version
  FROM presence
  GROUP BY 1,2
),
removals AS (
  SELECT
    'REMOVE_DEPENDENCY' AS event_type,
    ds.next_snapshot_at AS event_time,
    lp.dependent_artifact_id,
    lp.package_artifact_id,
    lp.last_version AS version_before,
    CAST(NULL AS varchar) AS version_after
  FROM last_presence lp
  JOIN dependent_snapshots_seq ds
    ON ds.dependent_artifact_id=lp.dependent_artifact_id
   AND ds.snapshot_at=lp.last_seen_at
  WHERE ds.next_snapshot_at IS NOT NULL
),
events_union AS (
  SELECT * FROM adds
  UNION ALL
  SELECT * FROM upgrades
  UNION ALL
  SELECT * FROM removals
),
enriched AS (
  SELECT
    e.*,
    ROW_NUMBER() OVER(
      PARTITION BY e.dependent_artifact_id,e.package_artifact_id,e.event_time
      ORDER BY b.snapshot_at DESC
    ) AS rn
  FROM events_union e
  LEFT JOIN monthly_base b
    ON b.dependent_artifact_id = e.dependent_artifact_id
   AND b.package_artifact_id = e.package_artifact_id
   AND b.snapshot_at <= e.event_time
)
SELECT
  event_type,
  event_time,
  dependent_artifact_id,
  package_artifact_id,
  version_before,
  version_after
FROM enriched
WHERE rn=1 OR rn IS NULL

""")

df2

,event_type,event_time,dependent_artifact_id,package_artifact_id,version_before,version_after
0,UPGRADE_DEPENDENCY,2025-01-31 11:43:05.000,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,2.7.16,2.7.13
1,ADD_DEPENDENCY,2024-10-26 05:51:23.000,oG67hEEmN/dyNJWaPzR3UZZKT4wrmtKkEagc3YgYECA=,n8E+uk9vWuFiiG+qTj7rscadRhCf1Ovf7yTmua72fBc=,<NA>,2.7.16
